In [1]:
import os
import sys
sys.path.append('../../')
from recs.backtest import get_backtest_answer
from recs.utils.path import *
import __init__
from recs.utils.common import predict_ranking, readjson2dict, recall_evaluate
import recs
from tqdm import tqdm
import pandas as pd
tqdm.pandas()

In [20]:
tran_data = pd.read_pickle(DATA_PATH / "trans_buy.pkl")
real = get_backtest_answer(tran_data, start=20200000, end=20210000)

100%|██████████| 28496/28496 [01:16<00:00, 374.07it/s]


In [34]:
target_date = 202000
trans_buy = tran_data[tran_data.buy_date < (target_date)*100]
crm = pd.read_pickle(DATA_PATH / "crm_diff.pkl")
target_users = crm[crm.yyyymm == 201912].id_number.unique().tolist()
best_sells = trans_buy[
    (trans_buy.buy_date > 20171200 )
].fund_id.value_counts().rename_axis('fund_id').reset_index(name='count')
trans_buy

,id_number,fund_id,buy_date,yyyymm
1,13627,930,20170327,201702
5,13651,748,20190709,201906
8,13651,2053,20170926,201708
11,13653,2518,20170616,201705
12,13654,2091,20160322,201602
...,...,...,...,...
395706,67934,3041,20180116,201712
395707,67951,2108,20191114,201910
395708,67969,482,20180509,201804
395709,67971,2058,20180515,201804


100%|██████████| 233507/233507 [02:50<00:00, 1372.57it/s]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [12]:
vaecf = recs.models.VAECF(
    k=10,
    autoencoder_structure=[20],
    act_fn="tanh",
    likelihood="mult",
    n_epochs=100,
    batch_size=100,
    learning_rate=0.001,
    beta=1.0,
    seed=123,
    use_gpu=True,
    verbose=True,
)
trans_buy['t_weight'] = trans_buy.buy_date.progress_apply(lambda x: pow((x - trans_buy.buy_date.min() + 1) / (trans_buy.buy_date.max() - trans_buy.buy_date.min()), 2))
rate_df = trans_buy.groupby(["id_number", "fund_id"]).agg({'t_weight':'sum'}).reset_index().rename(columns={"t_weight": "rating"})
uir_data = rate_df[['id_number', 'fund_id', 'rating']].values
train_set = recs.datasets.Dataset.from_uir(uir_data, seed=41)
df = pd.DataFrame(columns=['userId', 'itemId', 'rating'], data=uir_data)
pred = predict_ranking(vaecf, df, 'userId', 'itemId', 'rating', False)
vaecf.fit(train_set)



  0%|          | 0/100 [00:00<?, ?it/s]

,userId,itemId,rating
0,8.0,722.0,0.966552
1,8.0,1391.0,0.986233
2,10.0,1724.0,0.967247
3,10.0,2096.0,0.967247
4,10.0,2531.0,0.967247
...,...,...,...
175128,79842.0,2940.0,0.941823
175129,79842.0,3160.0,0.955152
175130,79843.0,2998.0,0.105557
175131,79846.0,3197.0,0.124976


In [70]:

pred.sort_values(by='rating', ascending=False, inplace=True)
pred

,userId,itemId,rating
33651181,29023.0,1914.0,4.135778e-01
50394219,43526.0,20.0,3.708725e-01
24709389,21473.0,20.0,3.641103e-01
51979197,44784.0,20.0,3.640985e-01
62014359,53522.0,20.0,3.541389e-01
...,...,...,...
88912613,76523.0,1019.0,3.072849e-15
69479009,59991.0,1019.0,3.068669e-15
70804751,61208.0,1019.0,3.035293e-15
5065685,4410.0,1019.0,3.035293e-15


In [71]:




rec_dict = {}
for u in tqdm(target_users):
    temp = pred[pred.userId == u]
    if len(temp) == 0:
#         ignore_items = trans_buy[trans_buy.id_number == u].fund_id.unique().tolist()        
#         rec_dict[u] = best_sells[~best_sells['fund_id'].isin(ignore_items)]['fund_id'].tolist()[:10]
        rec_dict[u] = best_sells['fund_id'].tolist()[:10]
    else:
        rec_dict[u] = temp.itemId.unique().tolist()[:10]

100%|██████████| 69839/69839 [41:43<00:00, 27.89it/s]


In [72]:
list(rec_dict.keys())[0]

38831

In [73]:
len(rec_dict)

69839

In [22]:
for i in tqdm(rec_dict):
    rec_dict[i] = [int(x) for x in rec_dict[i]]

100%|██████████| 69839/69839 [00:00<00:00, 476865.37it/s]


In [19]:
import json
from recs.utils.common import convert_result_json_2_csv
r_name = "rb_2020_vaecf"
json_name = r_name+".json"
json_save_path = RESULT_PATH / json_name 
# with open(json_save_path, 'w') as jf:
#     json.dump(rec_dict, jf)

csv_name = f"{r_name}.csv"
save_path = RESULT_PATH / csv_name
# convert_result_json_2_csv(str(json_save_path), str(save_path))

vaecf_data = pd.read_csv(save_path)

In [28]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from sklearn.cluster import Birch
import itertools
import numpy as np
def normalized(x):
    return (x - x.mean()) / x.std()

n_cluster = 100

user_data = crm[crm.yyyymm == 201912]
udata = user_data[['local_foreign_total', 'local_total',
       'local_demand_deposit', 'local_fixed_deposit', 'foreign_total',
       'foreign_fixed_deposit', 'foreign_demand_deposit', 'invest_type', 'age',
       'monthly_trade_vol', 'stock_inventory_val', 'KPI','local_foreign_total_diff', 'local_total_diff',
       'local_demand_deposit_diff', 'local_fixed_deposit_diff',
       'foreign_total_diff', 'foreign_fixed_deposit_diff',
       'foreign_demand_deposit_diff', 'monthly_trade_vol_diff',
       'stock_inventory_val_diff', 'local_foreign_total_pct',
       'local_total_pct', 'local_demand_deposit_pct',
       'local_fixed_deposit_pct', 'foreign_total_pct',
       'foreign_fixed_deposit_pct', 'foreign_demand_deposit_pct',
       'monthly_trade_vol_pct', 'stock_inventory_val_pct', 'age_group']] 
for i in ['local_foreign_total_diff', 'local_total_diff',
       'local_demand_deposit_diff', 'local_fixed_deposit_diff',
       'foreign_total_diff', 'foreign_fixed_deposit_diff',
       'foreign_demand_deposit_diff', 'monthly_trade_vol_diff',
       'stock_inventory_val_diff', 'local_foreign_total_pct',
       'local_total_pct', 'local_demand_deposit_pct',
       'local_fixed_deposit_pct', 'foreign_total_pct',
       'foreign_fixed_deposit_pct', 'foreign_demand_deposit_pct',
       'monthly_trade_vol_pct', 'stock_inventory_val_pct']:
    udata[i] = normalized(udata[i])
udata = udata.replace(-np.inf, -1)
udata.fillna(0, inplace=True)
x_embedded = TSNE(n_components=3, init='random', random_state=47, n_jobs=30).fit_transform(udata.values)
x_embedded = np.array(x_embedded)





/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [32]:
# x_embedded = udata.values
brc = Birch(n_clusters=n_cluster)
brc.fit(x_embedded)
brc_x = np.array(brc.predict(x_embedded))
user_data['cluster'] = brc_x

tran_data = trans_buy.copy()


cluster_rec = {}
for i in tqdm(range(n_cluster)):
    userlen = len(user_data[user_data['cluster'] == i])
    tranlen = len(tran_data[tran_data['id_number'].isin( user_data[user_data['cluster'] == i]['id_number'])])
    temp = tran_data[tran_data['id_number'].isin( user_data[user_data['cluster'] == i]['id_number'])]['fund_id'].value_counts().index.tolist()[:10]
    cluster_rec[i] = temp
    



cluster_rec_vae = {}
for i in tqdm(range(n_cluster)):
    count_dict = {}

    temp = vaecf_data[vaecf_data['uid'].isin(user_data[user_data['cluster'] == i]['id_number'])]

    for j in range(10):
        temp_count_dict = temp[f"fundid{j}"].value_counts().to_dict()
        for k in temp_count_dict.keys():
            if count_dict.get(k):
                count_dict[k] += int(temp_count_dict[k])
            else:
                count_dict[k] = int(temp_count_dict[k])


    result = dict(sorted(count_dict.items(), key=lambda item: item[1], reverse=True))    
    cluster_rec_vae[i] = list(result.keys())[0:10]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
100%|██████████| 100/100 [00:00<00:00, 132.06it/s]


In [35]:
cluster_rec_dict = {}
cvae_rec_dict = {}
for u in tqdm(target_users):
    temp = user_data[user_data['id_number']==u]
    t_cluster = temp.cluster.tolist()[0]    
    cluster_rec_dict[u] = cluster_rec[t_cluster]
    cvae_rec_dict[u] = cluster_rec_vae[t_cluster]
    

100%|██████████| 69839/69839 [00:36<00:00, 1910.76it/s]


In [87]:
# with tsne
print(recall_evaluate(cluster_rec_dict, real))
print(recall_evaluate(cvae_rec_dict, real))
print(recall_evaluate(rec_dict, real))

100%|██████████| 28496/28496 [00:00<00:00, 238334.87it/s]


{'recall@3': 0.02143453803047146, 'recall@5': 0.029408687632739832, 'recall@10': 0.045609663078775065, 'recall@all': 0.045609663078775065}


100%|██████████| 28496/28496 [00:00<00:00, 246018.87it/s]


{'recall@3': 0.015910294430033105, 'recall@5': 0.021342771416299304, 'recall@10': 0.03265360196428376, 'recall@all': 0.03265360196428376}


100%|██████████| 28496/28496 [00:00<00:00, 207472.72it/s]

{'recall@3': 0.020790596760610798, 'recall@5': 0.029351500893187235, 'recall@10': 0.049574975405781395, 'recall@all': 0.049574975405781395}


In [90]:
print(recall_evaluate(cluster_rec_dict, real))
print(recall_evaluate(cvae_rec_dict, real))
print(recall_evaluate(rec_dict, real))

100%|██████████| 28496/28496 [00:00<00:00, 199809.57it/s]


{'recall@3': 0.021067441066646975, 'recall@5': 0.029303800202788582, 'recall@10': 0.044927233944598646, 'recall@all': 0.044927233944598646}


100%|██████████| 28496/28496 [00:00<00:00, 227664.04it/s]


{'recall@3': 0.015433688113855584, 'recall@5': 0.02122427245261342, 'recall@10': 0.03270093747448047, 'recall@all': 0.03270093747448047}


100%|██████████| 28496/28496 [00:00<00:00, 174419.39it/s]

{'recall@3': 0.020790596760610798, 'recall@5': 0.029351500893187235, 'recall@10': 0.049574975405781395, 'recall@all': 0.049574975405781395}


In [3]:
tran_data = pd.read_pickle(DATA_PATH / "trans_buy.pkl")

In [4]:
tran_data

,id_number,fund_id,buy_date,yyyymm
1,13627,930,20170327,201702
3,13642,1838,20200820,202007
5,13651,748,20190709,201906
6,13651,1461,20200303,202002
7,13651,1579,20200707,202006
...,...,...,...,...
395707,67951,2108,20191114,201910
395708,67969,482,20180509,201804
395709,67971,2058,20180515,201804
395710,67971,2274,20200131,201912


In [10]:
user2020 = list(tran_data[tran_data.buy_date > 20200000].id_number.unique())

In [9]:
user2017 =list(tran_data[tran_data.buy_date < 20200000].id_number.unique())

In [16]:
len(user2020)

28496

In [17]:
len(user2017)

40906

In [22]:
new_user = list(set(user2020) - set(user2017))

In [24]:
new_real = {}
for i in tqdm(real):
    if i in new_user:
        new_real[i] = real[i]        
    

100%|██████████| 28496/28496 [00:42<00:00, 671.25it/s]


In [42]:
with open(json_save_path, "r") as jf:
    rec_dict = json.load(jf)

temp_dict = {}
for i in rec_dict:
    temp_dict[int(i)] = rec_dict[i]
    
print(recall_evaluate(cluster_rec_dict, new_real))
print(recall_evaluate(cvae_rec_dict, new_real))
print(recall_evaluate(temp_dict, new_real))

100%|██████████| 11056/11056 [00:00<00:00, 276265.24it/s]


{'recall@3': 0.00911768801828498, 'recall@5': 0.012117766208305285, 'recall@10': 0.018408138361224086, 'recall@all': 0.018408138361224086}


100%|██████████| 11056/11056 [00:00<00:00, 271117.60it/s]


{'recall@3': 0.007307484132285147, 'recall@5': 0.009563346285488113, 'recall@10': 0.013020408728740475, 'recall@all': 0.013020408728740475}


100%|██████████| 11056/11056 [00:00<00:00, 274744.64it/s]

{'recall@3': 0.00816423360969309, 'recall@5': 0.010793052772520937, 'recall@10': 0.013169624776164578, 'recall@all': 0.013169624776164578}


In [39]:
list(rec_dict.keys())[0]

'38831'

In [40]:
rec_dict['38831']

[2936.0,
 3215.0,
 2940.0,
 2717.0,
 3197.0,
 3041.0,
 2416.0,
 2518.0,
 1108.0,
 3072.0]